In [1]:
# https://blog.naver.com/ostin1038/222213027645

In [2]:
import numpy as np
import pandas as pd

In [3]:
d18 = pd.read_csv('data/question_12_18.csv')
d19 = pd.read_csv('data/question_12_19.csv')

In [4]:
D = pd.merge(d18, d19, on="com_code")
D.head()

,com_code,Revenue_18,OperatingIncomeLoss_18,Revenue_19,OperatingIncomeLoss_19
0,111847,9.954134e+11,4.223128e+10,9.602151e+11,2.693892e+10
1,152039,1.345482e+11,5.183868e+09,1.257976e+11,3.860651e+09
2,154055,NaN,NaN,NaN,NaN
3,114765,6.508736e+11,-1.214061e+10,6.155784e+11,-1.398375e+10
4,149804,1.110213e+12,6.690127e+09,1.284997e+12,3.129117e+10


In [5]:
D = D.dropna(axis=0, how="any")

In [13]:
# Q1
_ = """
문제1.) 다음중 19년 매출 기준으로 매출 상위 6번째까지 해당하는 기업의 
        19년 매출 총합이 19년 전체 매출 총합의 비중을 얼마만큼 차지하는가? 
"""

In [6]:
D

,com_code,Revenue_18,OperatingIncomeLoss_18,Revenue_19,OperatingIncomeLoss_19
0,111847,9.954134e+11,4.223128e+10,9.602151e+11,2.693892e+10
1,152039,1.345482e+11,5.183868e+09,1.257976e+11,3.860651e+09
3,114765,6.508736e+11,-1.214061e+10,6.155784e+11,-1.398375e+10
4,149804,1.110213e+12,6.690127e+09,1.284997e+12,3.129117e+10
5,174004,2.466138e+11,1.789772e+09,2.595855e+11,1.224458e+10
...,...,...,...,...,...
72,119140,6.009279e+11,1.050172e+10,6.176687e+11,1.868764e+10
73,164502,1.585857e+11,-5.769174e+08,1.667104e+11,3.354151e+09
74,1036446,1.850755e+12,1.202621e+10,1.976766e+12,2.187629e+10
75,1042775,5.664833e+12,1.974108e+11,5.981877e+12,2.185661e+11


In [7]:
D.Revenue_19.sort_values(ascending=False).iloc[0:6].sum() / D.Revenue_19.sum()

0.6895563009038589

In [8]:
# Q2
_ = """
문제2.) 18년과 19년 기업들의 매출이 통계적으로 유의미하게 차이가 나는지 
        대응표본 T검정을 통해 알아보고자 한다. 
        이를 위해 18년, 19년 매출에 해당하는 아웃라이어를 구하고 
        18년, 19년 둘 중 하나라도 아웃라이어에 해당하는 기업은 제외하고 수행한다. 

        다음 T검정을 수행해 P-value값을 소수 셋째자리까지 구하여라 

        (등분산 조건은 성립한다고 가정하고 진행하며 아웃라이어를 구할때 
        Tukey방식을 활용한다)

        표본은 총 58개이며 자유도는 57, 소수 넷째자리는 버림하시오) 
"""

In [9]:
q1, q3 = np.quantile(D.Revenue_18, [0.25, 0.75])
iqr = q3 - q1
lower18 = q1 - 1.5*iqr
upper18 = q3 + 1.5*iqr

In [10]:
q1, q3 = np.quantile(D.Revenue_19, [0.25, 0.75])
iqr = q3 - q1
lower19 = q1 - 1.5*iqr
upper19 = q3 + 1.5*iqr

In [11]:
df2 = D[
    (D.Revenue_18 >= lower18) &
    (D.Revenue_18 <= upper18) &
    (D.Revenue_19 >= lower19) &
    (D.Revenue_19 <= upper19) ]

In [12]:
from scipy.stats import ttest_rel

In [13]:
ttest_rel(df2.Revenue_18, df2.Revenue_19)

Ttest_relResult(statistic=-2.9574420032621687, pvalue=0.004508366641607686)

In [ ]:
# Q3
_ = """
문제3.) 군집 분석을 통해 유사 업체끼리 군집으로 분류하고자 한다. 
        군집 분석 방식은 K-means 방식을 활용하되 아래의 전처리를 수행하고 진행한다.
        (base table을 토대로 진행)

① 3개의 합성합수를 생성한다 
  매출(Revenue) 성장률, 영업이익(OperatingIncomeLoss) 성장률, 
  19년 영업이익률( OperatingIncomeLoss / Revenue) 
  (성장률은 18년 대비 19년 성장률)

② 19년 매출과 영업이익을 표준화한다
  (Hint: sklearn.preprocessing 라이브러리, StandardScaler 메소드 활용)

  위 3개 합성함수와 표준화한 2개 변수, 총 5개 변수를 바탕으로 
  K-means 군집분석을 수행한다.
  ( sklearn.cluster 라이브러리 KMeans 활용, 
  클러스터 수는 4개, Seed는 1636 나머지 디폴트) 


  그 후 클러스터별 매출 성장률 중앙값이 
  가장 큰 클러스터의 매출 성장률 중앙값을  
  소수 둘째자리까지 구하시오
  (셋째자리 버림) 
"""

In [34]:
df3_1 = pd.DataFrame({'revenue_growth': D.Revenue_19 / D.Revenue_18 - 1,
              'operation_growth': D.OperatingIncomeLoss_19 / D.OperatingIncomeLoss_18 - 1,
              'profit_rate': D.Revenue_19 / D.Revenue_18}).reset_index()
df3_1

,index,revenue_growth,operation_growth,profit_rate
0,0,-0.035361,-0.362110,0.964639
1,1,-0.065037,-0.255257,0.934963
2,3,-0.054228,0.151816,0.945772
3,4,0.157433,3.677216,1.157433
4,5,0.052599,5.841422,1.052599
...,...,...,...,...
59,72,0.027858,0.779483,1.027858
60,73,0.051232,-6.813918,1.051232
61,74,0.068087,0.819051,1.068087
62,75,0.055967,0.107164,1.055967


In [19]:
from sklearn.preprocessing import StandardScaler

In [33]:
scaler = StandardScaler()
scaled = scaler.fit_transform(D[['Revenue_19', 'OperatingIncomeLoss_19']])
df3_2 = pd.DataFrame(scaled, columns=['revenue19', 'operation19'])
df3_2

,revenue19,operation19
0,-0.078612,-0.084286
1,-0.253020,-0.163098
2,-0.150647,-0.224036
3,-0.010726,-0.069424
4,-0.225056,-0.134467
...,...,...
59,-0.150210,-0.112464
60,-0.244469,-0.164828
61,0.133866,-0.101575
62,0.971008,0.570114


In [36]:
df3 = pd.concat([df3_1, df3_2], axis=1)
df3

,index,revenue_growth,operation_growth,profit_rate,revenue19,operation19
0,0,-0.035361,-0.362110,0.964639,-0.078612,-0.084286
1,1,-0.065037,-0.255257,0.934963,-0.253020,-0.163098
2,3,-0.054228,0.151816,0.945772,-0.150647,-0.224036
3,4,0.157433,3.677216,1.157433,-0.010726,-0.069424
4,5,0.052599,5.841422,1.052599,-0.225056,-0.134467
...,...,...,...,...,...,...
59,72,0.027858,0.779483,1.027858,-0.150210,-0.112464
60,73,0.051232,-6.813918,1.051232,-0.244469,-0.164828
61,74,0.068087,0.819051,1.068087,0.133866,-0.101575
62,75,0.055967,0.107164,1.055967,0.971008,0.570114


In [26]:
from sklearn.cluster import KMeans

In [27]:
model = KMeans(n_clusters=4, random_state=1636)

In [37]:
model.fit(df3)

KMeans(n_clusters=4, random_state=1636)

In [38]:
model.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int32)

In [39]:
df3['label'] = model.labels_
df3

,index,revenue_growth,operation_growth,profit_rate,revenue19,operation19,label
0,0,-0.035361,-0.362110,0.964639,-0.078612,-0.084286,1
1,1,-0.065037,-0.255257,0.934963,-0.253020,-0.163098,1
2,3,-0.054228,0.151816,0.945772,-0.150647,-0.224036,1
3,4,0.157433,3.677216,1.157433,-0.010726,-0.069424,1
4,5,0.052599,5.841422,1.052599,-0.225056,-0.134467,1
...,...,...,...,...,...,...,...
59,72,0.027858,0.779483,1.027858,-0.150210,-0.112464,2
60,73,0.051232,-6.813918,1.051232,-0.244469,-0.164828,2
61,74,0.068087,0.819051,1.068087,0.133866,-0.101575,2
62,75,0.055967,0.107164,1.055967,0.971008,0.570114,2


In [41]:
df3.groupby(by='label').median()

,index,revenue_growth,operation_growth,profit_rate,revenue19,operation19
label,,,,,,
0,16.0,0.055458,-208.471987,1.055458,-0.232302,-0.165565
1,13.0,0.030051,0.199872,1.030051,-0.154253,-0.111496
2,64.0,0.051925,-0.284618,1.051925,-0.244469,-0.164817
3,43.5,0.025116,-0.285740,1.025116,-0.228762,-0.172094


In [43]:
# 블로그랑 답이 다름 label 0, revenue_growth 0.05
# 매출성장률 계산 방법이 다름...